<a href="https://colab.research.google.com/github/2bar/2bar.github.io/blob/master/Document_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Doc2vecを用いて、文章の類似性を調べて行きます。

今回は、よく似ていると言われている、太宰治と芥川龍之介について、どちらの作品でも精神を病んだ人物を取り扱っている「人間失格」との「河童」の類似性について調べることとします。


github上の表記は、文章が全て表示され、読字性が良くないので、文章の表示に関わる部分はコメントアウトしてあります。

今回は青空文庫のデータを読み込みます。

In [ ]:
import zipfile
import os.path, urllib.request as request

In [ ]:
URL1="https://www.aozora.gr.jp/cards/000035/files/301_ruby_5915.zip"
URL2="https://www.aozora.gr.jp/cards/000879/files/69_ruby_1321.zip"

次のコードは、プロジェクトフォルダに対象のファイルがなければダウンロードする内容です。 

まずは、人間失格をダウンロードします。

In [ ]:
localfile1 = "301_ruby_5915.zip"

if not os.path.exists(localfile1):
    print("ファイルをダウンロードします")
    request.urlretrieve(URL1,localfile1)

ファイルをダウンロードします


In [ ]:
import zipfile
zipfile = zipfile.ZipFile(localfile1, 'r')

In [ ]:
with zipfile.open('ningen_shikkaku.txt', 'r') as file:
  bindata = file.read()
  ningen_original = bindata.decode('shift_jis')

ダウンロードした文章の全文になります。

In [ ]:
# print(ningen_original)

読み込んだ小説のデータから、正規表現を用いてルビなどを削除します。

In [ ]:
import re
import pickle

ningen_original = re.split(r'\-{5,}',ningen_original)[2] # ヘッダーを取り除く
ningen_original = re.split(r'底本：',ningen_original)[0] # フッターを取り除く
ningen_original = ningen_original.strip()
# print(ningen_original)

In [ ]:
ningen = re.sub('《[^》]+》', '', ningen_original) # ルビの削除
ningen = re.sub('［[^］]+］', '', ningen) # 読みの注意の削除
ningen = re.sub('[｜ 　「」\n]', '', ningen) # | と全角半角スペース、「」と改行の削除
ningen = re.sub(r'一\r\r|\r', '', ningen) # 先頭のゴミを削除

seperator = '。'  # 。をセパレータに指定
ningen_list = ningen.split(seperator)  # セパレーターを使って文章をリストに分割する
ningen_list.pop() # 最後の要素は空の文字列になるので、削除
ningen_list = [x+seperator for x in ningen_list]  # 文章の最後に。を追加
        
# print(ningen_list)

with open('ningen_list.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(ningen_list, f)

形態素解析を用いて単語に分割します。
形態素解析ツールのjanomeを使いますが、最新版だと、動作が安定しないために、旧Verを使用します。

In [ ]:
pip install janome==0.3.9

     |████████████████████████████████| 25.1MB 173kB/s 


人間失格について、単語に分ける分かち書き処理を行います。

In [ ]:
from janome.tokenizer import Tokenizer
import pickle

t = Tokenizer()
w = []

for sentence in ningen_list:
  w.append(t.tokenize(sentence, wakati=True))
# print(w)
with open('ningen_list.pickle', mode='wb') as f:
  pickle.dump(w, f)

保存ができているか、確認します。

In [ ]:
with open('ningen_list.pickle', mode='rb') as f:
    ningen_list = pickle.load(f)

# print(ningen_list)

河童についても同様の工程を行います。

In [ ]:
localfile2 = "69_ruby_1321.zip"

if not os.path.exists(localfile2):
    print("ファイルをダウンロードします")
    request.urlretrieve(URL2,localfile2)

ファイルをダウンロードします


In [ ]:
import zipfile
zipfile2 = zipfile.ZipFile(localfile2, 'r')

In [ ]:
with zipfile2.open('kappa.txt', 'r') as file:
  bindata = file.read()
  kappa_original = bindata.decode('shift_jis')

In [ ]:
# print(kappa_original)

In [ ]:
import re
import pickle

kappa_original = re.split(r'\-{5,}',kappa_original)[2] 
kappa_original = re.split(r'底本：',kappa_original)[0] 
kappa_original = kappa_original.strip()
# print(kappa_original)

In [ ]:
kappa = re.sub('《[^》]+》', '', kappa_original) 
kappa = re.sub('［[^］]+］', '', kappa) 
kappa = re.sub('[｜ 　「」\n]', '', kappa) 
kappa = re.sub(r'一\r\r|\r', '', kappa) 

seperator = '。'  
kappa_list = kappa.split(seperator)  
kappa_list.pop() 
kappa_list = [x+seperator for x in kappa_list]  
        
# print(kappa_list)

with open('kappa_list.pickle', mode='wb') as f:  
    pickle.dump(kappa_list, f)

In [ ]:
from janome.tokenizer import Tokenizer
import pickle

t = Tokenizer()
w = []

for sentence in kappa_list:
  w.append(t.tokenize(sentence, wakati=True))
# print(w)
with open('kappa_list.pickle', mode='wb') as f:
  pickle.dump(w, f)

In [ ]:
with open('kappa_list.pickle', mode='rb') as f:
    kappa_list = pickle.load(f)

# print(kappa_list)

リストの次元削減を行います。

In [ ]:
import itertools

ningen_test = list(itertools.chain.from_iterable(ningen_list))
kappa_test = list(itertools.chain.from_iterable(kappa_list))
# print(ningen_test)
# print(kappa_test)

doc2vecを使って学習を行い、モデルを作成します。

文章のベクトル数値と類似度を表示します。

In [ ]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

training_docs = []

sent1 = TaggedDocument(ningen_test,
                       tags=["ningen"])
sent2 = TaggedDocument(kappa_test,
                       tags=["kappa"])


training_docs.append(sent1)
training_docs.append(sent2)

# min_count：学習に使う単語の最低出現回数
# epochs:epochs数
# dm：学習モデル=DM
model = Doc2Vec(documents=training_docs,
                epochs=50,
                min_count=1,
                dm=1)
                
print(model.docvecs['ningen'])
print(model.docvecs.most_similar('ningen'))

[ 3.617362   -3.545907    3.2165666  -0.8724458  -0.66895455  1.6487614
 -4.369904    1.0433578  -4.1704893  -0.19822484 -5.4874163  -0.9460287
  2.299591    2.7180586   0.45350903 -3.900706    6.51131    -0.10638342
 -4.3536944   3.2107437   3.584672    0.8027633   3.0520837   1.2013904
 -7.2905307   2.8166358  -0.1514414  -2.8700635   1.7030493  -2.9389184
 -1.3500183   2.355725    5.7104163  -1.2142012   3.549989   -2.948784
  4.0582066  -0.37098587 -0.83639807  1.2056462  -2.6199882  -5.46042
 -5.701296   -0.3353905  -0.92379564 -2.8409088  -2.2608638   5.6987796
  9.533841   -5.181526    0.8637443  -3.7449896   3.487731    2.987077
  2.4876752  -3.241227    4.536504   -4.5420456  -3.4150512  -1.7109405
 -2.63371    -2.09839     2.0499089   2.952258   -1.823846   -0.5542439
 -1.6471553  -0.84571296  4.535371   -4.824631    3.5617101  -0.55766386
  7.582061    3.3961515   1.5510861  -1.1228591  -3.5828302  -2.6931481
  2.166183   -5.0214305  -0.50704426  1.6432022  -0.109321    1.52

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


以上より、人間失格と河童の類似度は約0.25と高くはないので、類似性があるとされているのは単語や助詞など以外のファクターが大きいと考えられる。

参考URL:

gensimでDoc2Vec:
https://kento1109.hatenablog.com/entry/2017/11/15/181838

Doc2Vecによる文書ベクトル推論の安定化について:
https://buildersbox.corp-sansan.com/entry/2019/04/10/110000#f-d99c2ee8

Python と gensim で doc2vec を使う:
https://kitayamalab.wordpress.com/2016/11/14/python-%E3%81%A8-gensim-%E3%81%A7-doc2vec-%E3%82%92%E4%BD%BF%E3%81%86/

Pythonでflatten（多次元リストを一次元に平坦化）:
https://note.nkmk.me/python-list-flatten/